In [1]:
# Вопросы:
#   1. scoring - как работает, как происходит сравнение двух результатов?
#                Что означает, скажем, такой результат:
#
#    - genes_all
#      {'rf__max_depth': 8, 'rf__min_samples_leaf': 3, 'rf__n_estimators': 200}
#      {'AUC': 0.6956898656898656, 'Balanced Accuracy': 0.6292735042735041, 'Sensivity': 0.41777777777777786, 'Specificity': 0.8407692307692308}
#
#    - genes_all
#      {'rf__max_depth': 8, 'rf__min_samples_leaf': 11, 'rf__n_estimators': 200}
#      {'AUC': 0.6942796092796091, 'Balanced Accuracy': 0.6266178266178266, 'Sensivity': 0.5144444444444445, 'Specificity': 0.7387912087912089}
#
#    - ["CDK1", "FOXM1", "LRIG2", "PLK1", "RACGAP1", "RRM2", "TMPO"]
#      {'SVM__C': 181.01933598375618}
#      {'AUC': 0.7315201465201464, 'Balanced Accuracy': 0.6841849816849818, 'Sensivity': 0.6366666666666666, 'Specificity': 0.7317032967032968}
#
#    - ['CDK1', 'FOXM1', 'PLK1', 'RACGAP1', 'RRM2', 'TMPO']
#      {'SVM__C': 10.079368399158989}
#      {'AUC': 0.7411355311355311, 'Balanced Accuracy': 0.6983943833943833, 'Sensivity': 0.6377777777777777, 'Specificity': 0.7590109890109891}
#
#    - Reference (SVM): 
#      {'SVM__C': 16.0}
#      {'AUC': 0.721019536019536, 'Balanced Accuracy': 0.673397435897436, 'Sensivity': 0.6233333333333334, 'Specificity': 0.7234615384615385}
#
#   2. Как именно вычисляется AUC? (свободный член -inf; +inf)
#   3. Насколько надежна проверка RepeatedStratifiedKFold?   n_repeats > 100, std(scoring)
#      Нужно ли подвергать ее сомнению? 
#      Нужно ли как-то разбивать данные по группам?
#   4. Важны ли матожидания экспрессий с биологической точки зрения? (scaler)
#   5. Является ли набор TCGA.tsv однородным? (ответ: да)

In [7]:
import pandas as pd
import numpy as np

from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.preprocessing import *

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
import xgboost as xgb

from sklearn.tree  import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
# helper function - true-positive rate 
def TPR(y_true, y_pred):
    M = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = M[0, 0], M[0, 1], M[1, 0], M[1, 1]
    return TP / (TP + FN)

# helper function - true-negative rate 
def TNR(y_true, y_pred):
    M = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = M[0, 0], M[0, 1], M[1, 0], M[1, 1]
    return TN / (TN + FP)

# scoring values
scoring = {
    "AUC": "roc_auc",
    "Balanced Accuracy": "balanced_accuracy",
    "Sensivity": make_scorer(TPR),
    "Specificity": make_scorer(TNR),
}

In [4]:
# Load TCGA RNA-seq data
df = pd.read_csv("../../data/breast_cancer/TCGA.tsv", sep="\t", index_col=0)
print(("df shape = {}\n").format( df.shape) )
#print( df[0:10])


df shape = (111, 13519)



In [37]:
# Take gene subset (or all genes)
# reference scenario
genes = ["CDK1", "FOXM1", "LRIG2", "MSH2", "PLK1", "RACGAP1", "RRM2", "TMPO"]

# other variants
#genes = ["CDK1", "FOXM1", "LRIG2", "PLK1", "RACGAP1", "RRM2", "TMPO"]
#genes = ["SHOX2", "ETNPPL", "FGF14"]
# genes = list( df.columns.values)  # all available genes

# Form data
X = df[genes].to_numpy()
y = df["Class"].to_numpy()

In [45]:
# classification algo and params_for_opt
#--------------------------------------------------

# 1. SVM
#clfr = ("SVM", SVC(kernel="linear", class_weight="balanced"))
#opt_params = {"SVM__C": np.logspace(-4, 4, 9, base=4)}

# 2. Random Forest
#clfr = ("rf", RandomForestClassifier( class_weight="balanced" ))
#opt_params = {
#    "rf__n_estimators": [200, 400, 600],
#    "rf__max_depth": [10, 20],
#    "rf__min_samples_leaf": [7]
#}

# 3. Decision Tree
#clfr = ("dt", DecisionTreeClassifier( class_weight="balanced" ))
#opt_params = {
#    "dt__max_depth": [5, 10, 20]
#}

# 4. GBDT
clfr = ("gdt", GradientBoostingClassifier() )
opt_params = {
    "gdt__max_depth": [ 6 ],
    "gdt__n_estimators": [100]
}


In [46]:
# Establish a classification pipeline
scaler = StandardScaler()
classifier = Pipeline([
   ("scaler", scaler),
   clfr ])

In [47]:
# Optimization cylce
splitter = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)

CV = GridSearchCV( 
    classifier,
    opt_params,
    scoring=scoring,
    cv=splitter,
    refit=False)
CV.fit(X, y)

# Infer best parameter
mean_test_scoring_values = {s: CV.cv_results_["mean_test_" + s] for s in scoring}
max_BA_index = np.argmax(mean_test_scoring_values["Sensivity"])
best_scores = {s: mean_test_scoring_values[s][max_BA_index] for s in scoring}

# print best model parameters and score
#print( genes )
print( CV.cv_results_["params"][max_BA_index] )
print({s: mean_test_scoring_values[s][max_BA_index] for s in scoring})

{'gdt__max_depth': 6, 'gdt__n_estimators': 100}
{'AUC': 0.5154253154253153, 'Balanced Accuracy': 0.5250915750915751, 'Sensivity': 0.4, 'Specificity': 0.6501831501831503}
